In [1]:
###
#   FIG4.IPYNB:
#   Creating plots for Figures 2 and 3 in the paper.
###
# By Kirill Sechkar

from cistrans_model import *
import bokeh.io as bkio
bkio.output_notebook()

# set up jax
jax.config.update('jax_platform_name', 'cpu')
jax.config.update("jax_enable_x64", True)

Loading BokehJS ...

In [2]:
# INITIALISE THE MODEL AND THE SIMULATOR
# get default model parameter values
par = set_default_pars_cistrans()

# ODE term definition
cis_term = ODETerm(ode_cis)
trans_term = ODETerm(ode_trans)

In [3]:
 # INITIALISE THE RANGE OF INDUCTION LEVELS AND DISTURBANCE CHARACTERISTICS
# define extents of interference considered and magnitude of disturbance
indi_axis = jnp.logspace(-4+jnp.log10(1.8), 0, 21)
# indi_axis=jnp.array([0, 1e-1])
indi_axis_np = np.array(indi_axis)  # np format for robustness calculations
indi_range = (np.min(indi_axis_np), np.max(indi_axis_np)) # range of interference induction levels

# define the range of tested activator gRNA induction extents
inda_axis = jnp.logspace(-4+jnp.log10(1.8), 0, 21)
# inda_axis=jnp.array([1e-5, 1e-2])
inda_axis_np = np.array(inda_axis)  # np format for robustness calculations
inda_range = (np.min(inda_axis_np), np.max(inda_axis_np)) # range of activator induction levels

# RESOURCE DISTURBANCE
ind_c_default = 0.0 # default induction level of the competitor
ind_c_pert = 1.0    # induction level of the competitor during the perturbing pulse
tf_pre = (0, 24)  # simulation time frame BEFORE the competitor is induced induction
tf_pulse = (24, 26)  # simulation time frame DURING the competitor is induced induction
tf_post = (26, 50)  # simulation time frame AFTER the competitor is induced induction
pert_char_res = (tf_pre, tf_pulse, tf_post, ind_c_pert)

# TRANSCRIPTIONAL PERTURBATION
ind_a_pert_relative = 0.25    # induction level of the competitor during the perturbing pulse
pert_char_transc = (tf_pre, tf_pulse, tf_post, ind_a_pert_relative) # same time frames as for the resource disturbance

# settle time criterion
settle_bound=0.05

In [4]:
# RESOURCE DISTURBANCE FOR OPEN-LOOP SYSTEM
par_res_open = par.copy()
par_res_open['i_offtarget'] = 1.0

# generate meshgrid of induction levels
indi_mesh_open, inda_mesh_open = jnp.meshgrid(indi_axis, jnp.array([0])) # meshgird of interference and activator induction levels (no activator present)
indi_mesh_open_ravel = indi_mesh_open.ravel()
inda_mesh_open_ravel = inda_mesh_open.ravel()
stacked_mesh_open = jnp.stack((indi_mesh_open_ravel, inda_mesh_open_ravel), axis=1) # stack the meshgrid into a single array
mesh_open=jnp.concatenate((stacked_mesh_open, ind_c_default * jnp.ones((len(indi_mesh_open_ravel), 1))), axis=1) # add entries for competitor induction

# simulate
peakdists_res_open_jnp, peakdists_percent_res_open_jnp,\
    rectime_indices_res_open_jnp, \
    xs_res_open_jnp, ts_res_open_jnp = induction_to_dynmetrics_res(mesh_open, par_res_open, cis_term, pert_char_res,settle_bound=settle_bound)

# unpack the trajectories
xs_res_open=np.array(xs_res_open_jnp)
ts_res_open=np.array(ts_res_open_jnp)

# convert peak disturbance values to numpy format
peakdists_res_open=np.array(peakdists_res_open_jnp)
peakdists_percent_res_open=np.array(peakdists_percent_res_open_jnp)

# find recovery times
rectimes_res_open=ts_res_open.take(np.array(rectime_indices_res_open_jnp))-tf_post[0]

In [5]:
# RESOURCE DISTURBANCE FOR CIS FEEDBACK
par_res_open = par.copy()

# generate meshgrid of induction levels
indi_mesh_cis, inda_mesh_cis = jnp.meshgrid(indi_axis, jnp.array([0])) # meshgird of interference and activator induction levels (no activator present)
indi_mesh_cis_ravel = indi_mesh_cis.ravel()
inda_mesh_cis_ravel = inda_mesh_cis.ravel()
stacked_mesh_cis = jnp.stack((indi_mesh_cis_ravel, inda_mesh_cis_ravel), axis=1) # stack the meshgrid into a single array
mesh_cis=jnp.concatenate((stacked_mesh_cis, ind_c_default * jnp.ones((len(indi_mesh_cis_ravel), 1))), axis=1) # add entries for competitor induction
# simulate
peakdists_res_cis_jnp, peakdists_percent_res_cis_jnp,\
    rectime_indices_res_cis_jnp, \
    xs_res_cis_jnp, ts_res_cis_jnp = induction_to_dynmetrics_res(mesh_cis, par_res_open, cis_term, pert_char_res,settle_bound=settle_bound)

# unpack the trajectories
xs_res_cis=np.array(xs_res_cis_jnp)
ts_res_cis=np.array(ts_res_cis_jnp)

# convert peak disturbance values to numpy format
peakdists_res_cis=np.array(peakdists_res_cis_jnp)
peakdists_percent_res_cis=np.array(peakdists_percent_res_cis_jnp)

# find recovery times
rectimes_res_cis=ts_res_cis.take(np.array(rectime_indices_res_cis_jnp))-tf_post[0]

In [ ]:
# RESOURCE DISTURBANCE FOR TRANS FEEDBACK
par_res_trans = par.copy()

# generate meshgrid of induction levels
indi_mesh_trans, inda_mesh_trans = jnp.meshgrid(indi_axis, inda_axis) # meshgird of interference and activator induction levels
indi_mesh_trans_ravel = indi_mesh_trans.ravel()
inda_mesh_trans_ravel = inda_mesh_trans.ravel()
stacked_mesh_trans = jnp.stack((indi_mesh_trans_ravel, inda_mesh_trans_ravel), axis=1) # stack the meshgrid into a single array
mesh_trans=jnp.concatenate((stacked_mesh_trans, ind_c_default * jnp.ones((len(inda_mesh_trans_ravel), 1))), axis=1) # add entries for competitor induction

# simulate
peakdists_res_trans_jnp, peakdists_percent_res_trans_jnp,\
    rectime_indices_res_trans_jnp, \
    xs_res_trans_jnp, ts_res_trans_jnp = induction_to_dynmetrics_res(mesh_trans, par_res_trans, trans_term, pert_char_res,settle_bound=settle_bound)

# unpack the trajectories
xs_res_trans=np.array(xs_res_trans_jnp)
ts_res_trans=np.array(ts_res_trans_jnp)

# convert peak disturbance values to numpy format
peakdists_res_trans_ravel=np.array(peakdists_res_trans_jnp)
peakdists_res_trans=peakdists_res_trans_ravel.reshape(len(inda_axis), len(indi_axis)).T
peakdists_percent_res_trans_ravel=np.array(peakdists_percent_res_trans_jnp)
peakdists_percent_res_trans=peakdists_percent_res_trans_ravel.reshape(len(inda_axis), len(indi_axis)).T

# find recovery times
rectimes_res_trans_ravel=ts_res_trans.take(np.array(rectime_indices_res_trans_jnp))-tf_post[0]
rectimes_res_trans=rectimes_res_trans_ravel.reshape(len(inda_axis), len(indi_axis)).T

In [ ]:
# TRANSCRIPTIONAL PERTURBATION FOR OPEN LOOP
par_transc_open=par.copy()
par_transc_open['i_offtarget'] = 1.0

# simulate
peakdists_transc_open_jnp, peakdists_percent_transc_open_jnp,\
    rectime_indices_transc_open_jnp, \
    xs_transc_open_jnp, ts_transc_open_jnp = induction_to_dynmetrics_transc(mesh_cis, par_transc_open, cis_term, pert_char_transc,settle_bound=settle_bound)

# unpack the trajectories
xs_transc_open=np.array(xs_transc_open_jnp)
ts_transc_open=np.array(ts_transc_open_jnp)

# convert peak disturbance values to numpy format
peakdists_transc_open=np.array(peakdists_transc_open_jnp)
peakdists_percent_transc_open=np.array(peakdists_percent_transc_open_jnp)

# find recovery times
rectimes_transc_open=ts_transc_open.take(np.array(rectime_indices_transc_open_jnp))-tf_post[0]

In [ ]:
# TRANSCRIPTIONAL PERTURBATION FOR CIS FEEDBACK

# simulate
peakdists_transc_cis_jnp, peakdists_percent_transc_cis_jnp,\
    rectime_indices_transc_cis_jnp, \
    xs_transc_cis_jnp, ts_transc_cis_jnp = induction_to_dynmetrics_transc(mesh_cis, par, cis_term, pert_char_transc,settle_bound=settle_bound)

# unpack the trajectories
xs_transc_cis=np.array(xs_transc_cis_jnp)
ts_transc_cis=np.array(ts_transc_cis_jnp)

# convert peak disturbance values to numpy format
peakdists_transc_cis=np.array(peakdists_transc_cis_jnp)
peakdists_percent_transc_cis=np.array(peakdists_percent_transc_cis_jnp)

# find recovery times
rectimes_transc_cis=ts_transc_cis.take(np.array(rectime_indices_transc_cis_jnp))-tf_post[0]

In [ ]:
# TRANSCRIPTION PERTURBATION FOR TRANS FEEDBACK

# simulate
peakdists_transc_trans_jnp, peakdists_percent_transc_trans_jnp,\
    rectime_indices_transc_trans_jnp, \
    xs_transc_trans_jnp, ts_transc_trans_jnp = induction_to_dynmetrics_transc(mesh_trans, par, trans_term, pert_char_transc,settle_bound=settle_bound)

# unpack the trajectories
xs_transc_trans = np.array(xs_transc_trans_jnp)
ts_transc_trans = np.array(ts_transc_trans_jnp)

# convert peak disturbance values to numpy format
peakdists_transc_trans_ravel = np.array(peakdists_transc_trans_jnp)
peakdists_transc_trans = peakdists_transc_trans_ravel.reshape(len(inda_axis), len(indi_axis)).T
peakdists_percent_transc_trans_ravel = np.array(peakdists_percent_transc_trans_jnp)
peakdists_percent_transc_trans = peakdists_percent_transc_trans_ravel.reshape(len(inda_axis), len(indi_axis)).T

# find recovery times
rectimes_transc_trans_ravel = ts_transc_trans.take(np.array(rectime_indices_transc_trans_jnp)) - tf_post[0]
rectimes_transc_trans = rectimes_transc_trans_ravel.reshape(len(inda_axis), len(indi_axis)).T

In [ ]:
# FIG4D,E: THE TRADEOFFS IN RESPONSE TO THE TWO PERTURBATIONS

# Maximum fold-change in output protein level upon disturbance
tradeoff_peakdist_fold_fig_x_range = (1,128)
tradeoff_peakdist_fold_fig_y_range = (1,1.55)
tradeoff_peakdist_fold_fig = bkplot.figure(frame_width=256,
                                           frame_height=144,
                                           x_axis_label='Resource pert.: max. fold-change',
                                           y_axis_label='Transc. pert.: max. fold-change',
                                           x_axis_type='log', y_axis_type='log',
                                           x_range=tradeoff_peakdist_fold_fig_x_range,
                                           y_range=tradeoff_peakdist_fold_fig_y_range
                                           )
tradeoff_peakdist_fold_fig.output_backend = "svg"
# points
peakdists_fold_res_open= np.array(peakdists_percent_res_open/100+1)
peakdists_fold_res_cis= np.array(peakdists_percent_res_cis/100+1)
peakdists_fold_res_trans_ravel= np.array(peakdists_percent_res_trans_ravel/100+1)
peakdists_fold_transc_open= np.array(peakdists_percent_transc_open/100+1)
peakdists_fold_transc_cis= np.array(peakdists_percent_transc_cis/100+1)
peakdists_fold_transc_trans_ravel= np.array(peakdists_percent_transc_trans_ravel/100+1)
tradeoff_peakdist_fold_fig.circle(peakdists_fold_res_trans_ravel,
                                  peakdists_fold_transc_trans_ravel,
                                  size=4, alpha=0.5,
                                  color='#ff6700ff', legend_label='Trans feedback')
tradeoff_peakdist_fold_fig.cross(peakdists_fold_res_cis,
                                 peakdists_fold_transc_cis,
                                 size=6, alpha=1, line_width=2,
                                 color='#de3163ff', legend_label='Cis feedback')
tradeoff_peakdist_fold_fig.x(tradeoff_peakdist_fold_fig_x_range[0] * np.ones_like(peakdists_fold_transc_open),
                             peakdists_fold_transc_open,
                             size=8, alpha=1, line_width=2,
                             color='#48d1ccff', legend_label='Open loop')
# legend
tradeoff_peakdist_fold_fig.legend.border_line_color = "gray"
tradeoff_peakdist_fold_fig.legend.border_line_alpha = 0.3
tradeoff_peakdist_fold_fig.legend.background_fill_color = "white"
tradeoff_peakdist_fold_fig.legend.background_fill_alpha = 1
tradeoff_peakdist_fold_fig.legend.margin=2
tradeoff_peakdist_fold_fig.legend.padding=1
tradeoff_peakdist_fold_fig.legend.spacing=2
tradeoff_peakdist_fold_fig.legend.label_standoff=0
tradeoff_peakdist_fold_fig.legend.location = "bottom"
tradeoff_peakdist_fold_fig.legend.orientation = "horizontal"
tradeoff_peakdist_fold_fig.legend.visible = False
# ticks
tradeoff_peakdist_fold_fig.xaxis.ticker = [1, 2, 4, 8, 16, 32, 64]
tradeoff_peakdist_fold_fig.yaxis.ticker = [1, 1.2, 1.44]
# fonts
tradeoff_peakdist_fold_fig.title.text_font_size = "8pt"
tradeoff_peakdist_fold_fig.xaxis.axis_label_text_font_size = "8pt"
tradeoff_peakdist_fold_fig.yaxis.axis_label_text_font_size = "8pt"
tradeoff_peakdist_fold_fig.xaxis.major_label_text_font_size = "8pt"
tradeoff_peakdist_fold_fig.yaxis.major_label_text_font_size = "8pt"
tradeoff_peakdist_fold_fig.legend.label_text_font_size = "8pt"



# Recovery time after disturbance is removed
tradeoff_rectime_fig_x_range=(0,14.5)
tradeoff_rectime_fig_y_range=(0,4.5)
tradeoff_rectime_fig = bkplot.figure(frame_width=256,
                                     frame_height=144,
                                     x_axis_label='Resource pert.:'+str(int(settle_bound*100))+'% settling time, h',
                                     y_axis_label='Transc. pert.:'+str(int(settle_bound*100))+'% settling time, h',
                                     # x_axis_type='log', y_axis_type='log',
                                     x_range=tradeoff_rectime_fig_x_range,
                                     y_range=tradeoff_rectime_fig_y_range
                                     )
tradeoff_rectime_fig.output_backend = "svg"
# points
tradeoff_rectime_fig.circle(rectimes_res_trans_ravel,
                                  rectimes_transc_trans_ravel,
                                  size=4, alpha=0.5,
                                  color='#ff6700ff', legend_label='Trans feedback')
tradeoff_rectime_fig.cross(rectimes_res_cis,
                                 rectimes_transc_cis,
                                 size=6, alpha=1, line_width=2,
                                 color='#de3163ff', legend_label='Cis feedback')
tradeoff_rectime_fig.x(tradeoff_rectime_fig_x_range[0] * np.ones_like(rectimes_transc_open),
                             rectimes_transc_open,
                             size=8, alpha=1, line_width=2,
                             color='#48d1ccff', legend_label='Open loop')
# legend
tradeoff_rectime_fig.legend.border_line_color = "gray"
tradeoff_rectime_fig.legend.border_line_alpha = 0.3
tradeoff_rectime_fig.legend.background_fill_color = "white"
tradeoff_rectime_fig.legend.background_fill_alpha = 1
tradeoff_rectime_fig.legend.margin=2
tradeoff_rectime_fig.legend.padding=1
tradeoff_rectime_fig.legend.spacing=2
tradeoff_rectime_fig.legend.label_standoff=0
tradeoff_rectime_fig.legend.location = "bottom"
tradeoff_rectime_fig.legend.orientation = "horizontal"
tradeoff_rectime_fig.legend.visible = False
# ticks
tradeoff_rectime_fig.xaxis.ticker = [0,3,6,9,12,15]
tradeoff_rectime_fig.yaxis.ticker = [0,1,2,3,4,5]
# fonts
tradeoff_rectime_fig.title.text_font_size = "8pt"
tradeoff_rectime_fig.xaxis.axis_label_text_font_size = "8pt"
tradeoff_rectime_fig.yaxis.axis_label_text_font_size = "8pt"
tradeoff_rectime_fig.xaxis.major_label_text_font_size = "8pt"
tradeoff_rectime_fig.yaxis.major_label_text_font_size = "8pt"
tradeoff_rectime_fig.legend.label_text_font_size = "8pt"


bkio.show(bklayouts.gridplot([[tradeoff_peakdist_fold_fig, tradeoff_rectime_fig]]))